In [ ]:
'''
NOTE: 
This pipeline couldn't manage merged observations.
Which would be developed in the near future.


Steps:
###### ensure point sources position and size ######
#### position
1. extract as many point sources using cheese as possible.
1.1 emlfill
2. comb merge the mos1, mos2, pn images,
3. blink check the missing point sources, add or sub the point sources ra,dec
#### size
3.1 create the new bkgregtdet/sky.fits using region commands
4. region and makemask command
# check the result if all the point sources size and radec are nice!

###### determine the cutoff flux or SB by logN-S #####
5. divide the region file into individual files
6. extract spectra from the region, make logN-logS plot
7. determine the excluded point sources cutoff flux

###### Final region mask #######
8. make new region file, last time check the position and the size
## crosscheck with eckert ##
9. edit the emllist.fits, *-bkgregtdet/sky.fits 
10. rerun the masks

'''

In [ ]:
import os
import numpy as np
from glob import glob
from astropy.io import fits
import pandas as pd
from tqdm import tqdm


In [ ]:
# define homepath
# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction'
# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID9647/reduction'
# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID828/reduction'
# homepath = '/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460'
os.chdir(f'{homepath}')

# 0 Exclude chips in the events
check set_chips_on.txt

In [ ]:
from glob import glob
import os

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG4936')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        # Read the content from the text file
        with open("set_chips_on.txt", "r") as file:
            content = file.read()

        # Extract values from the content
        m1on_values = content.split('"')[1].split()
        m2on_values = content.split('"')[3].split()

        # Find the indices where values are 'F' for M1ON and M2ON
        mos1_f_indices = [i + 1 for i, value in enumerate(m1on_values) if value == "F"]
        mos2_f_indices = [i + 1 for i, value in enumerate(m2on_values) if value == "F"]

        #### !! NOTE: PN chips are always not excluded!
        exchip_dir = {'mos1':mos1_f_indices, 'mos2':mos2_f_indices, 'pn':[]}
        express = f'''filtertype=expression imagebinning='imageSize' imagedatatype='Int32' squarepixels=yes ignorelegallimits=yes withxranges=yes withyranges=yes xcolumn='X' ximagesize=900 ximagemax=48400 ximagemin=3401 ycolumn='Y' yimagesize=900 yimagemax=48400 yimagemin=3401 updateexposure=yes filterexposure=yes verbosity=4'''

        f = open('exchip_inevt.sh', 'w')
        name_dir = {'mos1':'mos1*-allevc', 'mos2':'mos2*-allevc', 'pn':'pn*-allevc'}
        for name in ['mos1', 'mos2', 'pn']: # , 'mos2S002'
            add_express = ''
            input_evtfile = glob(f'{name_dir[name]}.fits')[0]
            oot_evtfile = glob(f'pn*-allevcoot.fits')[0]
            output_evtfile = f'{name}-allevc-exchips.fits'
            output_imgfile = f'{name}-allevc-exchips_img.fits'
            if len(exchip_dir[name]) > 0:
                for chip in exchip_dir[name]:
                    add_express += f'&&(CCDNR.ne.{chip})' 
                
            if 'mos' in name:
                f.write(f'''evselect table={input_evtfile}:EVENTS withfilteredset=yes filteredset={output_evtfile} expression="(#XMMEA_EM)&&(PATTERN<=12)&&((FLAG & 0x766aa000) == 0){add_express}" {express} imageset={output_imgfile} \n''')
            else:
                f.write(f'''evselect table={input_evtfile}:EVENTS withfilteredset=yes filteredset={output_evtfile} expression="(#XMMEA_EP)&&(PATTERN<=4)&&((FLAG & 0xb0000) == 0)&&((FLAG & 0x766aa000) == 0){add_express}" {express} imageset={output_imgfile} \n''')
                f.write(f'''evselect table={oot_evtfile}:EVENTS withfilteredset=yes filteredset=pn-allevcoot-exchips.fits expression="(#XMMEA_EP)&&(PATTERN<=4)&&((FLAG & 0xb0000) == 0)&&((FLAG & 0x766aa000) == 0){add_express}" {express} imageset={name}-oot-allevc-exchips_img.fits \n''')

        f.close()

# 1 extract as many point sources using cheese as possible


In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        mos1evt = glob(f'mos1*allevc-exchips.fits')[0]
        mos2evt = glob(f'mos2*allevc-exchips.fits')[0]
        pnevt = glob(f'pn*allevc-exchips.fits')[0]
        pnootevt = glob(f'pn*allevcoot-exchips.fits')[0]
        f = open('cheese.sh', 'w')
        f.write(f'''
source set_sas.txt
elo=350 
ehi=2000
cheese mos1file='{mos1evt}' mos2file='{mos2evt}' pnfile='{pnevt}' pnootfile='{pnootevt}' elowlist=$elo ehighlist=$ehi scale=0.5 mlmin=1 dist=1. ratetotal=3e-6 keepinterfiles=no 2>&1 | tee cheese_max.log
        '''
        )
        f.close()

In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('emlfill.sh', 'w')
        f.write(f'emlfill emlin=emllist.fits emlout=emllist_filled.fits')
        f.close()

# 2 comb merge the mos1, mos2, pn images,


In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('comb_img.sh', 'w')

        elo = 350
        ehi = 2000

        name_dir = {'mos1':'mos1*-allevc', 'mos2':'mos2*-allevc', 'pn':'pn*-allevc'}

        express = f'(PATTERN<=4)&&((FLAG & 0xb0000) == 0)&&((FLAG & 0x762a097c) == 0)&&(PI in [{elo}:{ehi}])&&(DETY in [-16510:14345])'
        plot_express = f'''imagebinning='imageSize' imagedatatype='Int32' squarepixels=yes ignorelegallimits=yes withxranges=yes withyranges=yes xcolumn='X' ximagesize=900 ximagemax=48400 ximagemin=3401 ycolumn='Y' yimagesize=900 yimagemax=48400 yimagemin=3401'''

        name_lst = []
        prefix_lst = ''
        for file in glob(f'*-cheeset.fits'):
            prefix = file.split('-')[0]
            name_lst.append(prefix)
            if 'mos' in prefix:
                prefix_lst += f"{prefix[-5:]}  "
            else:
                prefix_lst += f"{prefix[-4:]}"
        for name in name_lst:
            f.write(f'''source set_sas.txt \n''')
            f.write(f'''evselect table={name[0:-4]}-allevcoot-exchips.fits withimageset=Y {plot_express} expression='{express}' imageset='{name}-fovimootsky-{elo}-{ehi}.fits'  \n''')
            f.write(f'mv {name}-fovimt.fits {name}-fovimsky-{elo}-{ehi}.fits\n')
            f.write(f'mv {name}-fovimtexp.fits {name}-expimsky-{elo}-{ehi}.fits\n')
        f.write(f'''combimage prefixlist="{prefix_lst}" withcheese=yes cheesetype=t elowlist={elo} ehighlist={ehi} 2>&1 | tee combimg.log\n''')
        f.write(f'''mv comb-fovimsky-{elo}-{ehi}.fits comb-fovimsky-{elo}-{ehi}_masked.fits\n''')
        f.write(f'''mv comb-expimsky-{elo}-{ehi}.fits comb-expimsky-{elo}-{ehi}_masked.fits\n''')
        f.write(f'''combimage prefixlist="{prefix_lst}" withcheese=no elowlist={elo} ehighlist={ehi} 2>&1 | tee -a combimg.log\n''')
        f.close()

In [ ]:
# convert the emllist.fits to region file with fixed circle of 15“
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
    
        if os.path.exists('emllist_filled.fits'):
            with fits.open('emllist_filled.fits') as data_f:
                data = data_f[1].data

            f = open('emllist_filled.reg', 'w')
            f.write('''fk5\n''')

            for i in range(len(data['ID_INST'])):
                if data[i]['ID_INST']==0:
                    f.write(f'''circle({data[i]['RA']}, {data[i]['DEC']}, 15")\n''')
            f.close()
        else:
            print(homepath)


# 3 First check of pointsources regions, include as many 'likely' point sources as possible.

3.0 Blink check the emllist_filled with the comb image!
and save to emllist_every.fits!

In [ ]:
also need to exclude the cores. Just every suspicious pointsources!

or the bkg regions later would be problematic. 

3.1 create the new bkgregtdet/sky.fits using region commands

In [ ]:
# create emllist_every.fits, include all the possible point sources
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('emllist_every.reg'):
            filename = f'emllist_filled.fits'
            os.system(f'cp {filename} emllist_every.fits')

            f = fits.open(filename)
            dat = f[1].data
            f.close()
            dat['FLAG'] = 0

            # load the styles of the recarray data 
            sample_dype = dat.dtype
            sample = dat[0]

            # load the regions of complete pointsources
            srcname = f'emllist_every.reg'
            regfile = open(srcname)
            lines = regfile.readlines()
            regfile.close()

            num = len(lines)-3
            dat = dat[:num]

            for i, f in enumerate(lines[3:]):
                ra = f.split('(')[-1].split(',')[0]
                dec = f.split(',')[1]
                dat[i]['RA'] = ra
                dat[i]['DEC'] = dec
                dat[i]['ML_ID_SRC'] = int(i+1)
                dat[i]['DIST_NN'] = 1000
                dat[i]['DET_ML'] = 20
                dat[i]['BOX_ID_SRC'] = int(i+1)
                dat[i]['ID_INST'] = 0
                dat[i]['ID_BAND'] = 0
                dat[i]['ID_CLUSTER'] = int(i+1)



            newf = fits.open(f'emllist_every.fits', mode='update')
            newf['SRCLIST'].data = dat
            newf.flush()
            newf.close()
        else:
            print(homepath)



# 4 1st run of region 

In [ ]:
# ## if the SCTS is NULL in emllist_every.fits, the makemask won't work properly
# ## UPDATE: SCTS can't solve the problem!
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('emllist_every.reg'):
            f = fits.open('emllist_every.fits')
            dat = f[1].data
            for i in range(len(dat)):
                if ~np.isfinite(dat[i]['SCTS']):
                    dat[i]['SCTS'] = 300

            os.system('cp emllist_every.fits emllist_every_filled.fits')
            newf = fits.open('emllist_every_filled.fits', mode='update')
            newf[1].data = dat
            newf.flush()


In [ ]:
## edit region files
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('emllist_every.reg'):
            name_lst = []
            for file in glob(f'*allevc-exchips.fits'):
                prefix = file.split('-')[0]
                name_lst.append(prefix)
            
            with open(f'makemask_1st.sh', 'w') as f:
                f.write('''
source set_sas.txt
mkdir cheese_old
elo=350
ehi=2000
''')
                for prefix in name_lst:
                    f.write(f'''

mv {prefix}-cheeset.fits cheese_old
mv {prefix}-bkgregtdet.fits cheese_old
mv {prefix}-bkgregtsky.fits cheese_old

region eventset={prefix}-allevc-exchips.fits srclisttab=emllist_every_filled.fits:SRCLIST operationstyle=batch bkgfraction=0.5 radiusstyle=contour srcidcol=ML_ID_SRC shrinkconfused=no nosrcellipse=no nobkgellipse=no fovbkgannulus=no outunit=xy verbosity=4 regionset=sources_{prefix}_sky_srcreg.fits bkgregionset=sources_{prefix}_sky_bkgreg.fits | tee region-sky-{prefix}_2nd.log
region eventset={prefix}-allevc-exchips.fits srclisttab=emllist_every_filled.fits:SRCLIST operationstyle=batch bkgfraction=0.5 radiusstyle=contour srcidcol=ML_ID_SRC shrinkconfused=no nosrcellipse=no nobkgellipse=no fovbkgannulus=no outunit=detxy verbosity=4 regionset=sources_{prefix}_det_srcreg.fits bkgregionset=sources_{prefix}_det_bkgreg.fits | tee region-det-{prefix}_2nd.log
                        ''')


        else:
            print(homepath)

In [ ]:
## convert the *bkgregtsky.fits to region file
# the physical coordinates in mos1S001,mos2S002,pnS003-bkgregtsky should be the same
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_mos1_sky_srcreg.fits'):
            f = open('sources_every_srcreg_phy.reg','w')
            f.write('physical\n')

            data_f = fits.open('sources_mos1_sky_srcreg.fits')

            for i in range(1, len(data_f)):
                dat = data_f[i].data
                if np.isfinite(dat['R'][0][0]) & (dat['R'][0][0]>0):
                    f.write(f'''ellipse({dat['X'][0][0]},{dat['Y'][0][0]},{dat['R'][0][0]},{dat['R'][0][1]},{dat['ROTANG'][0][0]})\n''')
                else:
                    f.write(f'''ellipse({dat['X'][0][0]},{dat['Y'][0][0]},200,200,0)\n''')     

            f.close()
        else:
            print(homepath)



In [ ]:
## convert the *bkgregtsky.fits to region file
# the physical coordinates in mos1S001,mos2S002,pnS003-bkgregtsky should be the same
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_mos1_sky_srcreg.fits'):
            f = open('sources_every_bkgreg_phy.reg','w')
            f.write('physical\n')

            data_f = fits.open('sources_mos1_sky_bkgreg.fits')

            for i in range(1,len(data_f)) :
                dat = data_f[i].data
                if len(dat) > 1:
                    if np.isfinite(dat[0]['R'][0]) & (dat['R'][0][0]>0):
                        f.write(f'''annulus({dat[0]['X'][0]},{dat[0]['Y'][0]},{dat[0]['R'][0]},{dat[0]['R'][1]})\n''')
                    else:
                        f.write(f'''annulus({dat[0]['X'][0]},{dat[0]['Y'][0]},600,600)\n''')     
                    for j in range(1, len(dat)):
                        if np.isfinite(dat[j]['R'][0]) & (dat['R'][0][0]>0):
                            f.write(f'''-ellipse({dat[j]['X'][0]},{dat[j]['Y'][0]},{dat[j]['R'][0]},{dat[j]['R'][1]},{dat[j]['ROTANG'][0]})\n''')
                        else:
                            f.write(f'''-ellipse({dat[j]['X'][0]},{dat[j]['Y'][0]},250,250,0)\n''')
                else:
                    f.write(f'''annulus({dat[0]['X'][0]},{dat[0]['Y'][0]},250,800)\n''')
                    f.write(f'''-ellipse({dat[0]['X'][0]},{dat[0]['Y'][0]},250,250,0)\n''')
            f.close()
        else:
            print(homepath)

# 5 check, resize some point sources regions, 
NOTE: and save to fk5 and physical coords 
(phy coords are the same for every inst)

5.1 inspect sources_every_src/bkgreg_phy.reg!

In [ ]:
datapath = f'/Users/eusracenorth/Downloads'
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
# ids = ['12349', '3460', '828', '9647']
with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    ids = dat['ID']
# ids = ['4936', '9695']

for i in range(len(ids)):
    subdir = f"{rootdir}/SDSSTG{ids[i]}"
    savepath = f'{rootdir}/figs/srcs_every'
    os.makedirs(savepath, exist_ok = True)
    if os.path.exists(subdir):
        os.chdir(subdir)
        try:
            os.system(f"rm {savepath}/SDSSTG{ids[i]}.png")
            os.system(f'ds9 comb-fovimsky-350-2000.fits -region sources_every_srcreg_phy.reg -scale log -cmap b -smooth yes -colorbar no -zoom to fit &')
            os.system(f'sleep 5')
            os.system(f"xpaset -p ds9 saveimage png {savepath}/SDSSTG{ids[i]}.png")
            os.system(f"xpaset -p ds9 exit")

        except:
            print(subdir)



5.2 separate the regions to small regions

In [ ]:
# separate the edited regions to individual small region
## for srcreg
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_mos1_sky_srcreg.fits'):
            f = open('sources_every_srcreg_phy.reg')
            lines = f.readlines()[1:]
            f.close()

            os.makedirs('srcs_regions', exist_ok=True)

            for i, l in enumerate(lines):
                newf = open(f'srcs_regions/src_{int(i+1)}.reg', 'w')
                newf.write(f'&&((X,Y) IN {l.strip()})')
                newf.close()

In [ ]:
## for bkgreg
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_mos1_sky_srcreg.fits'):
            f = open('sources_every_bkgreg_phy.reg')
            lines = f.readlines()
            f.close()

            os.makedirs('srcs_regions', exist_ok=True)
            i = 0

            for l in enumerate(lines[1:]):
                if 'annu' in l[1]: 
                    newf.close()
                    i+=1
                    newf = open(f'srcs_regions/bkg_{int(i)}.reg', 'w')
                    newf.write(f'&&((X,Y) IN {l[1].strip()})')
                elif 'ellip' in l[1]:
                    newf.write(f'&&!((X,Y) IN {l[1][1:].strip()})')

            newf.close()
    

# 6 extract spectra from source and bkg regions

In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_every_bkgreg_phy.reg'):
            sumf = open('sources_spec.sh', 'w')

            for name in ['mos1', 'mos2', 'pn']:
                regdir = f'srcs_regions'
                regs = glob(f'{regdir}/src_*.reg')
                
                f = open(f'src_spec_{name}.sh', 'w')
                for reg in regs:

                    sf = open(reg)
                    regtxt = sf.readlines()[0]
                    sf.close()

                    if 'mos' in name:
                        f.write(f'\n')
                        f.write(f'evselect table={name}-allevc-exchips.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset={reg.split(".")[0]}_{name}_img.fits withspectrumset=Y spectrumset={reg.split(".")[0]}_{name}_spec.pi energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="(FLAG==0) && (PATTERN<=12) {regtxt}"  | tee {reg.split(".")[0]}_{name}_spec.log\n')
                    else:
                        f.write(f'\n')
                        f.write(f'evselect table={name}-allevc-exchips.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset={reg.split(".")[0]}_{name}_img.fits withspectrumset=Y spectrumset={reg.split(".")[0]}_{name}_spec.pi energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="(FLAG==0) && (PATTERN<=4) {regtxt}"  | tee {reg.split(".")[0]}_{name}_spec.log\n')
                f.close()
                sumf.write(f'''sh src_spec_{name}.sh\n''')
                

            sumf.close()

        else:
            print(homepath)

In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    # print(homepath)
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_every_bkgreg_phy.reg'):

            sumf = open('sources_bkg_spec.sh', 'w')

            for name in ['mos1', 'mos2', 'pn']:
                regdir = f'srcs_regions'
                regs = glob(f'{regdir}/bkg*.reg')
                
                f = open(f'bkg_spec_{name}.sh', 'w')
                for reg in regs:

                    sf = open(reg)
                    regtxt = sf.readlines()[0]
                    sf.close()

                    if 'mos' in name:
                        f.write(f'\n')
                        f.write(f'evselect table={name}-allevc-exchips.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset={reg.split(".")[0]}_{name}_img.fits withspectrumset=Y spectrumset={reg.split(".")[0]}_{name}_spec.pi energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="(FLAG==0) && (PATTERN<=12) {regtxt}"  | tee {reg.split(".")[0]}_{name}_spec.log\n')
                    else:
                        f.write(f'\n')
                        f.write(f'evselect table={name}-allevc-exchips.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset={reg.split(".")[0]}_{name}_img.fits withspectrumset=Y spectrumset={reg.split(".")[0]}_{name}_spec.pi energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="(FLAG==0) && (PATTERN<=4) {regtxt}"  | tee {reg.split(".")[0]}_{name}_spec.log\n')
                f.close()
                sumf.write(f'''sh bkg_spec_{name}.sh\n''')
                
            sumf.close()

# 7 determine the cutoff flux

7.1 cal pointsource region area

In [ ]:
#### create separate region file in fk5
## 1. open sources_every_bkgreg_phy.reg	sources_every_srcreg_phy.reg on mos1-allevc-exchips_img.fits in ds9 and convert phy to fk5
## 2. separate the sources*fk5.reg file to individual files 



7.1.1 open sources_every_bkgreg_phy.reg	sources_every_srcreg_phy.reg in ds9 and convert phy to fk5

In [ ]:
#### NOTE: Never run src regs and bkg regs together!!! #### 


import os
from astropy.io import fits

# datapath = '/Users/eusracenorth/Downloads'
# with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
#     dat = f[1].data
#     ids = dat['ID']

rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
ids = ['21128']
for i in range(len(ids)):
    subdir = f"{rootdir}/SDSSTG{ids[i]}"
    savepath = f'{rootdir}/figs/srcs_every_phy'
    os.makedirs(savepath, exist_ok=True)
    if os.path.exists(subdir):
        os.chdir(subdir)
        
        ##### src regs ######
        os.system(f'rm sources_every_srcreg_fk5.reg')
        os.system(f'rm {savepath}/SDSSTG{ids[i]}_srcreg.png')
        os.system(f'ds9 mos2-allevc-exchips_img.fits -region sources_every_srcreg_phy.reg -scale log -cmap b -smooth yes -colorbar no -zoom to fit &')
        os.system('sleep 5')
        os.system(f"xpaset -p ds9 regions system wcs")
        os.system(f"xpaset -p ds9 regions sky fk5")
        os.system(f"xpaset -p ds9 regions skyformat degrees")
        os.system(f"xpaset -p ds9 regions save sources_every_srcreg_fk5.reg")
        os.system(f"xpaset -p ds9 saveimage png {savepath}/SDSSTG{ids[i]}_srcreg.png")
        os.system("xpaset -p ds9 exit")

        # ###### bkg regs #########
        # os.system(f'rm sources_every_bkgreg_fk5.reg')
        # os.system(f'rm {savepath}/SDSSTG{ids[i]}_bkgreg.png')
        # os.system(f'ds9 mos2-allevc-exchips_img.fits -region sources_every_BKGreg_phy.reg -scale log -cmap b -smooth yes -colorbar no -zoom to fit &')
        # os.system('sleep 5')
        # os.system(f"xpaset -p ds9 regions system wcs")
        # os.system(f"xpaset -p ds9 regions sky fk5")
        # os.system(f"xpaset -p ds9 regions skyformat degrees")
        # os.system(f"xpaset -p ds9 regions save sources_every_bkgreg_fk5.reg")
        # os.system(f"xpaset -p ds9 saveimage png {savepath}/SDSSTG{ids[i]}_bkgreg.png")
        # os.system("xpaset -p ds9 exit")


In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG3128')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_every_srcreg_fk5.reg'):

            os.chdir(f'{homepath}')
            # separate the edited regions to individual small region
            ## for srcreg
            f = open('sources_every_srcreg_fk5.reg')
            lines = f.readlines()[3:]
            
            if len(lines)>100:
                print(homepath)
                print(len(lines))
            f.close()

            for i, l in enumerate(lines):
                newf = open(f'srcs_regions/fk5_src_{int(i+1)}.reg', 'w')
                newf.write(f'fk5\n')
                newf.write(f'{l.strip()}\n')
                newf.close()

        else:
            print(homepath)


In [ ]:

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG3128')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_every_bkgreg_fk5.reg'):

            os.chdir(f'{homepath}')
            ## for bkgreg
            f = open('sources_every_bkgreg_fk5.reg')
            lines = f.readlines()
            f.close()
            if len(lines)>300:
                print(homepath)
                print(len(lines))
            i = 0

            for l in enumerate(lines[3:]):
                if 'annu' in l[1]: 
                    newf.close()
                    i+=1
                    newf = open(f'srcs_regions/fk5_bkg_{int(i)}.reg', 'w')
                    newf.write(f'fk5\n')
                    newf.write(f'{l[1].strip()}\n')
                elif 'ellip' in l[1]:
                    newf.write(f'{l[1].strip()}\n')
            # newf.close()
        else:
            print(homepath)
            

7.1.2 check if sources_every_*reg_fky.reg exist

In [ ]:
import os

def find_directories_without_file(root_directory, target_file):
    missing_directories = []

    for subdir in os.listdir(root_directory):
        subdirectory_path = os.path.join(root_directory, subdir)
        file_path = os.path.join(subdirectory_path, target_file)

        if os.path.isdir(subdirectory_path) and not os.path.exists(file_path):
            missing_directories.append(subdirectory_path)

    return missing_directories

root_directory = rootdir
target_file = "sources_every_bkgreg_fk5.reg"

missing_directories = find_directories_without_file(root_directory, target_file)

if missing_directories:
    print("Subdirectories without the file:")
    lst = []
    for directory in missing_directories:
        if 'TG' in directory:
            lst.append(directory.split('TG')[-1])
            
else:
    print("All subdirectories have the file.")
print(lst)


7.1.3 check if sources_every_*reg_fk5.reg has content

In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')
srclst = []
bkglst = []
for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        if os.path.exists('sources_every_srcreg_fk5.reg'):
            ## for bkgreg
            f = open('sources_every_bkgreg_fk5.reg')
            lines = f.readlines()
            f.close()
            bkg_num = len(lines)
            f = open('sources_every_srcreg_fk5.reg')
            lines = f.readlines()
            
            src_num = len(lines)

            if (src_num<5):
                srclst.append(homepath.split('TG')[-1])
                print('src')
                print(homepath)
            if (bkg_num<5):
                bkglst.append(homepath.split('TG')[-1])
                print('bkg')
                print(homepath)
        else: print(f'{homepath} do not have sources*every*fk5.reg !')
print(f'src: {lst}')
print(f'bkg: {lst}')

In [ ]:
import os

def check_for_string_in_file(root_directory, target_file, search_string):
    missing_directories = []

    for subdir in os.listdir(root_directory):
        subdirectory_path = os.path.join(root_directory, subdir)
        file_path = os.path.join(subdirectory_path, target_file)

        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                file_content = file.read()
                if search_string not in file_content:
                    missing_directories.append(subdirectory_path)

    return missing_directories

root_directory = rootdir
target_file = "sources_every_bkgreg_fk5.reg"
search_string = "annu"

missing_directories = check_for_string_in_file(root_directory, target_file, search_string)

if missing_directories:
    print("Subdirectories without the string 'annu' in the file:")
    for directory in missing_directories:
        print(directory)
else:
    print("All subdirectories have the string 'annu' in the file.")


In [ ]:
#### create mask log
import pandas as pd

# f = open(f'{datapath}/cal_regionarea.sh', 'w')
CDELT = 6.94E-04 * 60 # arcmin per pixel

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG4936')
srclst = []
bkglst = []
for homepath in homepaths:
    os.chdir(homepath)
    if os.path.exists('sources_every_srcreg_fk5.reg'):
        if os.path.isdir(f'{homepath}/srcs_regions'):
            os.chdir(f'{homepath}/srcs_regions')

            srcfiles = glob('fk5_src*reg')
            num = len(srcfiles)
            newf = open('cal_areaxsec.sh', 'w')


            for pref in ['mos1', 'mos2', 'pn']:

                for i in range(1,num+1):
                    srcreg = f'fk5_src_{int(i)}.reg'
                    bkgreg = f'fk5_bkg_{int(i)}.reg'

                    # newf.write(f'rm {srcreg.split(".")[0]}_{name}_msk.fits\n')
                    # newf.write(f'rm {bkgreg.split(".")[0]}_{name}_msk.fits\n')
                    # newf.write(f'rm {srcreg.split(".")[0]}_{name}_msk_exp.fits\n')
                    # newf.write(f'rm {bkgreg.split(".")[0]}_{name}_msk_exp.fits\n')
                    # newf.write(f'rm {srcreg.split(".")[0]}_{name}_msk.log\n')
                    # newf.write(f'rm {bkgreg.split(".")[0]}_{name}_msk.log\n')

                    namefile = glob(f'../{pref}*-expimsky-350-2000.fits')[0]
                    name = namefile.split('/')[-1].split('-')[0]
                    newf.write(f'ftimgcalc {srcreg.split(".")[0]}_{name}_msk.fits \'regfilter("{srcreg}",A.P1,A.P2) ? (1):(0)\' a=../{name}-expimsky-350-2000.fits clobber=yes\n')
                    newf.write(f'ftimgcalc {bkgreg.split(".")[0]}_{name}_msk.fits \'regfilter("{bkgreg}",A.P1,A.P2) ? (1):(0)\' a=../{name}-expimsky-350-2000.fits clobber=yes\n')

                    newf.write(f'farith {srcreg.split(".")[0]}_{name}_msk.fits ../{name}-expimsky-350-2000.fits {srcreg.split(".")[0]}_{name}_msk_exp.fits "*"\n')
                    newf.write(f'farith {bkgreg.split(".")[0]}_{name}_msk.fits ../{name}-expimsky-350-2000.fits {bkgreg.split(".")[0]}_{name}_msk_exp.fits "*"\n')

                    newf.write(f'fimgstat {srcreg.split(".")[0]}_{name}_msk_exp.fits threshlo=I threshup=I > {srcreg.split(".")[0]}_{name}_msk.log\n')
                    newf.write(f'fimgstat {bkgreg.split(".")[0]}_{name}_msk_exp.fits threshlo=I threshup=I > {bkgreg.split(".")[0]}_{name}_msk.log\n')

            newf.close()      


In [ ]:
### make area csv ###
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')
srclst = []
bkglst = []

for i, homepath in enumerate(homepaths):
    if os.path.exists(f'{homepath}/srcs_regions'):
        os.chdir(f'{homepath}/srcs_regions')
        matching_files = glob('*msk.log')
        if matching_files:
            srcfiles = glob('fk5_src*reg')
            num = len(srcfiles)
            try:
                for pref in ['mos1', 'mos2', 'pn']:
                    namefile = glob(f'../{pref}*-expimsky-350-2000.fits')[0]
                    name = namefile.split('/')[-1].split('-')[0]
                    dir = {}
                    dir['srcidx'] = np.arange(1,num+1)
                    dir['srcarea[pixel]'] = np.zeros(int(num))
                    dir['bkgarea[pixel]'] = np.zeros(int(num))

                    for i in range(1,num+1):
                        srcreg = f'fk5_src_{int(i)}.reg'
                        bkgreg = f'fk5_bkg_{int(i)}.reg'
                        
                        f = open(f'{srcreg.split(".")[0]}_{name}_msk.log')
                        lines = f.readlines()[:]
                        f.close()
                        for line in lines:
                            if 'sum' in line:
                                dir['srcarea[pixel]'][int(i-1)] = float(line.split('=')[-1])

                        f = open(f'{bkgreg.split(".")[0]}_{name}_msk.log')
                        lines = f.readlines()[:]
                        f.close()
                        for line in lines:
                            if 'sum' in line:
                                dir['bkgarea[pixel]'][int(i-1)] = float(line.split('=')[-1])

                    df = pd.DataFrame(dir)
                    df.to_csv(f'sources_areasxsec_{pref}.csv', index=False)
                        # backexp="{bkgtxt}"
                    
                    #### tidy csv to dats dir ####
                    datpath = f'{rootdir}/dats/logNS'
                    srcname = homepath.split('/')[-1]
                    df.to_csv(f'{datpath}/{srcname}_sources_areasxsec_{pref}.csv', index=False)
            except:
                print(homepath)
        else:
            print(homepath)
            continue


In [ ]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG*')
srclst = []
bkglst = []
for homepath in homepaths:
    os.chdir(homepath)
    if os.path.exists('sources_every_srcreg_fk5.reg'):
        if os.path.isdir(f'{homepath}/srcs_regions'):
            os.chdir(f'{homepath}/srcs_regions')

            f = open(f'gen_rmf.sh', 'w')

            specfiles = glob('src_1_*_spec.pi')

            for specfile in specfiles:
                
                if 'mos1' in specfile:
                    badpix_file = f'../mos1S001-fovimsky-350-8000.fits'
                elif 'mos2' in specfile:
                    badpix_file = f'../mos2S002-fovimsky-350-8000.fits'
                else:
                    badpix_file = f'../pnS003-fovimsky-350-8000.fits'

                f.write(f'''
            rmfgen spectrumset={specfile} rmfset={specfile.split('.')[0]}.rmf detmaptype=flat extendedsource=yes badpixlocation={badpix_file}\n
                    ''') 

            f.close()


            # arfgen spectrumset=source_{inst}_{obs}_bkg_spec_bin5.fits {set3} rmfset=source_{inst}_{obs}_bkg.rmf arfset=source_{inst}_{obs}_bkg.arf badpixlocation={inst}_hiband_loband_clean_fov.fits\n
            #     for reg in ['oofov']:#['reg1', 'reg2', 'reg3', 'reg4', 'reg5']:
            #         f.write(f'''
            # rmfgen spectrumset=source_{inst}_{obs}_{reg}_spec_bin.fits rmfset=source_{inst}_{obs}_{reg}.rmf detmaptype=flat extendedsource=yes badpixlocation={inst}_hiband_loband_clean_fov.fits\n
            #         ''')# arfgen spectrumset=source_{inst}_{obs}_{reg}_spec.fits {set3} rmfset=source_{inst}_{obs}_{reg}.rmf arfset=source_{inst}_{obs}_{reg}.arf badpixlocation={inst}_hiband_loband_clean_fov.fits\n

            # f = open(f'gen_arf_rmf_{obs}.sh')
            # print(f.readlines())

# 8 determine the excluded point sources cutoff flux

get counts rate from the spectrum

In [ ]:
def get_ctr(PATH):
    os.chdir(PATH)
    os.getcwd()
    
    srcspec_set = glob('src_*_*_spec.pi')
    bkgspec_set = glob('bkg_*_*_spec.pi')
    num = len(glob('src_*_mos1_spec.pi'))

    totctr_dict = {}
    srcctr_dict = {}

    for j, name in enumerate(['mos1', 'mos2', 'pn']):
        # read the s* arcmin2
        areafile = glob(f'sources_areasxsec_{name}.csv')[0]
        df = pd.read_csv(areafile)
        srcarea = df['srcarea[pixel]']
        bkgarea = df['bkgarea[pixel]']

        src_cts = np.zeros(int(num))
        bkg_cts = np.zeros(int(num))

        for i in range(1, num+1):
            srcspec = f'src_{i}_{name}_spec.pi'
            bkgspec = f'bkg_{i}_{name}_spec.pi'
            if (srcspec in srcspec_set) & (bkgspec in bkgspec_set):
                f = fits.open(f'{srcspec}')
                src_spec = f[1].data
                src_cts[int(i-1)] = np.sum(src_spec['COUNTS'])

                f = fits.open(f'{bkgspec}')
                bkg_spec = f[1].data
                bkg_cts[int(i-1)] = np.sum(bkg_spec['COUNTS'])     
            else:
                src_cts[int(i-1)] = 0
                bkg_cts[int(i-1)] = 0

        srcctr = src_cts/(srcarea)
        srcctr[~np.isfinite(srcctr)] = np.nan
        bkgctr = bkg_cts/(bkgarea)
        bkgctr[~np.isfinite(bkgctr)] = np.nan
        ### in order cores have been included in bkg region
        bkgnewctr = np.where(bkgctr > (2/3*srcctr), np.nanmedian(bkgctr), bkgctr)
        totctr_dict[name] = srcctr-bkgnewctr
        srcctr_dict[name] = srcctr
        
    totctr_dict['pn'] = totctr_dict['pn']/13.55 * 4.907
    srcctr_dict['pn'] = srcctr_dict['pn']/13.55 * 4.907
    return totctr_dict, srcctr_dict

In [ ]:
import matplotlib.pyplot as plt

def plot_logN_logS(data_df, savepath, savefile, detect_lim):
    fig, axs = plt.subplots(1, 3, figsize=(15, 4)) 
    for name in ['mos1', 'mos2', 'pn']: # 
        if 'pn' in name:
            axs[0].plot(np.arange(len(data_df[name])), data_df[name]/13.55 * 4.907, alpha = 0.5, label = name)
        else:
            axs[0].plot(np.arange(len(data_df[name])), data_df[name], alpha = 0.5, label = name)

    v = np.nanmax(data_df, axis=1)
    v1 = np.nansum(data_df, axis=1)
    axs[0].plot(np.arange(len(data_df[name])), v, 'r--', label = 'max')
    axs[0].plot(np.arange(len(data_df[name])), v1, 'b--', label = 'sum')
    axs[0].legend()


    
    hist, bin = np.histogram(v, bins = np.logspace(-6, np.log10(np.nanmax(v)), 30))
    axs[1].stairs(hist, bin, label = 'max', color ='r', linestyle = '--')
    axs[2].stairs(np.cumsum(hist[::-1])[::-1], bin, label = 'max', color ='r', linestyle = '--')
    hist, bin = np.histogram(v1, bins = np.logspace(-6, np.log10(np.nanmax(v)), 30))
    axs[1].stairs(hist, bin, label = 'sum', color = 'b')
    axs[2].stairs(np.cumsum(hist[::-1])[::-1], bin, label = 'sum', color ='b')

    axs[1].legend()
    axs[1].set_yscale('log')
    axs[1].axvline(detect_lim, c = 'r', linestyle = 'dashed')
    axs[0].axhline(detect_lim, c = 'r', linestyle = 'dashed')
    axs[2].axvline(detect_lim, c = 'r', linestyle = 'dashed')

    axs[1].axvline(detect_lim, c = 'b')
    axs[0].axhline(detect_lim, c = 'b')
    axs[2].axvline(detect_lim, c = 'b')

    axs[1].set_title('hist')
    axs[1].set_xscale('log')
    axs[1].set_ylabel('N')
    axs[1].set_xlabel('S (bg subtracted) [cts/s/pixel]')
    axs[0].set_yscale('log')
    axs[0].set_xlabel('src idx')
    axs[0].set_ylabel('S (bg subtracted) [cts/s/pixel]')

    axs[2].set_title('logN-logS')
    axs[2].set_xscale('log')
    axs[2].set_yscale('log')
    axs[2].set_ylabel('N')
    axs[2].set_xlabel('S (bg subtracted) [cts/s/pixel]')
    axs[2].legend()

    plt.savefig(f'{savepath}/{savefile}.png')
    plt.close()

8.1 give a dummy detection limit, plot all sources and view all

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


#### basic parameters ####
flux2cts = [4.907, 4.907, 13.55] # thin
sens = [1.15e-6, 1.28e-6, 1.78e-6]

#### save and plot ctr ####
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
homepaths = glob(f'{rootdir}/SDSSTG9695')


figpath = f'{rootdir}/figs/logNS'
datpath = f'{rootdir}/dats/logNS'
os.makedirs(figpath, exist_ok=True)
os.makedirs(datpath, exist_ok=True)

fin_files = glob(f'{datpath}/*')
fin_names = np.array([fin_file.split('_')[0] for fin_file in fin_files])
all_names = np.array([homepath.split('/')[-1] for homepath in homepaths])
unfin_names = np.setdiff1d(all_names, fin_names)
print(unfin_names)

srclst = []
bkglst = []
for srcname in unfin_names:
    os.chdir(f'{rootdir}/{srcname}')
    print(f'{rootdir}/{srcname}')
    if os.path.exists('sources_every_srcreg_fk5.reg'):
        if os.path.isdir(f'{rootdir}/{srcname}/srcs_regions'):
            totctr_dict,srcctr_dict = get_ctr(f'{rootdir}/{srcname}/srcs_regions') 
            df1 = pd.DataFrame(totctr_dict)
            df1.to_csv(f'{datpath}/{srcname}_totctr.csv')
            df2 = pd.DataFrame(srcctr_dict)
            df2.to_csv(f'{datpath}/{srcname}_srcctr.csv')

            plot_logN_logS(df1, f'{figpath}/totctr', f'{srcname}_totctr', 1.3e-6)
            plot_logN_logS(df2, f'{figpath}/srcctr', f'{srcname}_srcctr', 2e-5)


8.2 plot the assigned detetction limit on
and get ctr and area from the dats dir instead of srcs_regions dir

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


#### basic parameters ####
flux2cts = [4.907, 4.907, 13.55] # thin
sens = [1.15e-6, 1.28e-6, 1.78e-6]

#### load detection limits ####
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

#### save and plot ctr ####
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
homepaths = glob(f'{rootdir}/SDSSTG*')
figpath = f'{rootdir}/figs/logNS/final'
datpath = f'{rootdir}/dats/logNS'
os.makedirs(figpath, exist_ok=True)
os.makedirs(datpath, exist_ok=True)
all_names = [homepath.split('/')[-1] for homepath in homepaths]
srclst = []
bkglst = []
for srcname in all_names:
    msk = dat_f['SRCID']==srcname
    if np.sum(msk)>0:
        detect_lim = np.array(dat_f['detection_limit'])[msk][0]
        if os.path.exists(f'{rootdir}/{srcname}/sources_every_srcreg_fk5.reg'):
            if os.path.isdir(f'{rootdir}/{srcname}/srcs_regions'):
                totctr_dict = pd.read_csv(f'{datpath}/{srcname}_totctr.csv')
                plot_logN_logS(df1, f'{figpath}', f'{srcname}_totctr', detect_lim)
    else:
        print(srcname)


# 9 edit the emllist_final.fits
1) throw away the point sources under the cutoff limit 
cutoff limit is 3.0e-6 cts/s/pixel in mos, 7e-6 in pn
2) last adjust the size
3) check the result

9.1 throw away the point sources under the cutoff limit

In [ ]:

rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

# read detection limit


for i, ID in enumerate(dat_f['SRCID']): 
    detect_lim = dat_f['detection_limit'][i]
    ctr_f = pd.read_csv(f'{datpath}/{ID}_totctr.csv')
    vmax = np.nanmax(ctr_f, axis=1)
    msk = vmax>detect_lim

    sel_idx = list(np.arange(len(vmax))[msk])
    os.chdir(f'{rootdir}/{ID}')

    print(f'{rootdir}/{ID}')
    with open(f'sources_every_srcreg_fk5.reg') as f:
        lines = f.readlines()[3:]

    with open(f'sources_final_sum_srcreg_fk5.reg', 'w') as newf:
        newf.write('global color=cyan dashlist=8 3 width=2 font="helvetica 12 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
        newf.write('fk5\n')
        
        for i in range(len(sel_idx)):
            newf.write(lines[sel_idx[i]])


9.2 Last check the sum*final*reg! -> sum*final*edit.reg

In [ ]:
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
datapath = f'{rootdir}/dom'


with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    doc_ids = dat['ID']

all_ids = np.array([f.split('/')[-1].split('G')[-1] for f in glob(f'{rootdir}/SDSS*')]).astype(int)
# fin_ids = np.array([f.split('/')[-1].split('.')[0].split('G')[-1] for f in glob(f'{rootdir}/figs/srcs_final_resize_05/*.png')]).astype(int)
miss_ids = all_ids[~np.isin(all_ids, doc_ids)]

ids = miss_ids
# ids = [22635]
for i in range(len(ids)):
    subdir = f"{rootdir}/SDSSTG{ids[i]}"
    savepath = f'{rootdir}/figs/srcs_final_resize_05'
    os.makedirs(savepath, exist_ok = True)
    if os.path.exists(subdir):
        os.chdir(subdir)
        os.system(f"rm {savepath}/SDSSTG{ids[i]}.png")
        os.system(f'ds9 comb-fovimsky-350-2000.fits -region sources_final_sum_srcreg_fk5.reg -region emllist_every.reg -region R500_01_fk5.reg -scale log -cmap b -smooth yes -colorbar no -zoom to fit &')
        os.system(f'sleep 5')
        os.system(f"xpaset -p ds9 saveimage png {savepath}/SDSSTG{ids[i]}.png")
        os.system(f"xpaset -p ds9 exit")




APPEN check the missing ids

In [ ]:
with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    doc_ids = dat['ID']
all_ids = np.array([f.split('/')[-1].split('G')[-1] for f in glob(f'{rootdir}/SDSS*')]).astype(int)
fin_ids = np.array([f.split('/')[-1].split('.')[0].split('G')[-1] for f in glob(f'{rootdir}/figs/srcs_final_resize_05/*.png')]).astype(int)
miss_ids = all_ids[~np.isin(all_ids, fin_ids)]
docmiss_ids = all_ids[~np.isin(all_ids, doc_ids)]
print(miss_ids)
print(docmiss_ids)
print(len(doc_ids))

9.3 resize the regions

In [ ]:
# read ctr from csv, compare with detect lim
resize_scale = 0.5
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

ids = dat_f['SRCID']

for i, ID in enumerate(ids): 
    detect_lim = dat_f['detection_limit'][i]
    ctr_f = pd.read_csv(f'{datpath}/{ID}_totctr.csv')
    vmax = np.nanmax(ctr_f, axis=1)
    msk = vmax>detect_lim

    sel_idx = list(np.arange(len(vmax))[msk])
    os.chdir(f'{rootdir}/{ID}')

    print(f'{rootdir}/{ID}')
    with open(f'sources_every_srcreg_fk5.reg') as f:
        lines = f.readlines()[3:]
        

    with open(f'sources_final_sum_srcreg_fk5.reg', 'w') as newf:
        newf.write('global color=cyan dashlist=8 3 width=2 font="helvetica 12 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
        newf.write('fk5\n')
        
        for i in range(len(sel_idx)):
            line = lines[sel_idx[i]]
            r1 = float(line.split(',')[-2][:-1]) * resize_scale
            r2 = float(line.split(',')[-3][:-1]) * resize_scale
            line_segs = line.split(',')[:]
            newline = f'{line_segs[0]},{line_segs[1]},{r1}",{r2}",{line_segs[-1]}'
            newf.write(newline)


# 10 2nd and last run of region command

if consider region command in cheese, 
this is actually the 3rd time to run region

In [ ]:
# create emllist_final.fits, to convert the centers of fk5 coords to detxy and xy coords
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    filename = f'emllist_final.fits'
    os.system(f'cp emllist.fits {filename}')

    f = fits.open(filename)
    dat = f[1].data
    f.close()
    dat['FLAG'] = 0

    # load the styles of the recarray data 
    sample_dype = dat.dtype
    sample = dat[0]

    # load the regions of complete pointsources
    srcname = f'sources_final_sum_srcreg_fk5_edit.reg'
    regfile = open(srcname)
    lines = regfile.readlines()[:]
    regfile.close()

    num = len(lines)-3
    dat = dat[:num]

    for i, f in enumerate(lines[3:]):
        if len(f) > 1:
            ra = f.split('(')[-1].split(',')[0]
            dec = f.split(',')[1]
            dat[i]['RA'] = ra
            dat[i]['DEC'] = dec
            dat[i]['ML_ID_SRC'] = int(i+1)
            dat[i]['DIST_NN'] = 1000
            dat[i]['DET_ML'] = 20
            dat[i]['BOX_ID_SRC'] = int(i+1)
            dat[i]['ID_INST'] = 0
            dat[i]['ID_BAND'] = 0
            dat[i]['ID_CLUSTER'] = int(i+1)
        else:
            continue

    newf = fits.open(f'{filename}', mode='update')
    newf['SRCLIST'].data = dat
    newf.flush()
    newf.close()


In [ ]:
## edit region files

# create emllist_final.fits, to convert the centers of fk5 coords to detxy and xy coords
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    os.makedirs('cheese_old', exist_ok=True)
    f = open(f'region_2nd.sh', 'w')

    f.write(
    '''
source set_sas.txt
export M1=mos1
export M2=mos2
export PN=pn
export elo=350
export ehi=8000

for name in $M1 $M2 $PN
do

region eventset=${name}-allevc-exchips.fits operationstyle=global srclisttab=emllist_final.fits:SRCLIST expression="(ID_INST == 0)&&(DET_ML >= 1)" bkgregionset=${name}-bkgregtdet.fits bkgfraction=0.1 radiusstyle=contour outunit=detxy verbosity=4 2>&1 | tee region-det-${name}_2nd.log
region eventset=${name}-allevc-exchips.fits operationstyle=global srclisttab=emllist_final.fits:SRCLIST expression="(ID_INST == 0)&&(DET_ML >= 1)" bkgregionset=${name}-bkgregtsky.fits bkgfraction=0.1 radiusstyle=contour outunit=xy verbosity=4 2>&1 | tee region-sky-${name}_2nd.log

mv ${name}-bkgregtdet.fits ${name}-bkgregtdet_coords.fits
mv ${name}-bkgregtsky.fits ${name}-bkgregtsky_coords.fits

done

    '''
    )

    f.close()

In [ ]:
# alter the radius in *-bkgregtdet.fits and *-bkgregtsky.fits
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    as2det = 20 # 1arcsec to detxy or xy

    for name in ['mos1', 'mos2', 'pn']:
        fname_det = f'{name}-bkgregtdet_coords.fits'
        fname_sky = f'{name}-bkgregtsky_coords.fits'
        
        for filename in [fname_det, fname_sky]:
            os.system(f'''cp {filename} {filename.split('_')[0]}.fits''')

            f = fits.open(filename)
            dat = f[1].data
            f.close()

            # load the styles of the recarray data 
            sample_dype = dat.dtype
            sample = dat[0]

            # load the regions of complete pointsources
            srcname = f'sources_final_sum_srcreg_fk5_edit.reg'
            regfile = open(srcname)
            lines = regfile.readlines()[:]
            regfile.close()

            num = len(lines)-3
            dat = dat[:num]

            for i, f in enumerate(lines[3:]):
                if len(f) > 1:
                    if 'circle' in f:
                        r = float(f.split(',')[-1].split(')')[0][:-1]) * as2det
                        dat[int(i)]['R'] = [r, r, 0, 0]
                    elif 'ellipse' in f:
                        r2 = float(f.split(',')[-2][:-1]) * as2det
                        r1 = float(f.split(',')[-3][:-1]) * as2det
                        rot = float(f.split(',')[-1].split(')')[0][:-1])
                        dat[int(i)]['R'] = [r1, r2, 0, 0]
                        dat[int(i)]['ROTANG'] = [rot, 0, 0, 0]

                    else:
                        raise ValueError('Wrong shape, check the region file!')
                else:
                    continue
            newf = fits.open(f'''{filename.split('_')[0]}.fits''', mode='update')
            newf['REGION'].data = dat
            newf.flush()
            newf.close()


In [ ]:
## convert the *bkgregtsky.fits to region file
# the physical coordinates in mos1S001,mos2S002,pnS003-bkgregtsky should be the same
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    for name in ['mos1', 'mos2', 'pn']:
        data_f = fits.open(f'{name}-bkgregtsky.fits')
        dat = data_f[1].data

        f = open(f'{name}-bkgregtsky.reg','w')
        f.write('physical\n')
        for i in range(len(dat)):
            f.write(f'''ellipse({dat[i]['X'][0]},{dat[i]['Y'][0]},{dat[i]['R'][0]},{dat[i]['R'][1]},{dat[i]['ROTANG'][0]})\n''')
        f.close()



10.1 final check the regions! 

In [ ]:
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
datapath = f'{rootdir}/dom'


with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    doc_ids = dat['ID']

all_ids = np.array([f.split('/')[-1].split('G')[-1] for f in glob(f'{rootdir}/SDSS*')]).astype(int)
# fin_ids = np.array([f.split('/')[-1].split('.')[0].split('G')[-1] for f in glob(f'{rootdir}/figs/srcs_final_resize_05/*.png')]).astype(int)
miss_ids = all_ids[~np.isin(all_ids, doc_ids)]

# ids = all_ids
ids = ['40241']
for i in range(len(ids)):
    subdir = f"{rootdir}/SDSSTG{ids[i]}"
    savepath = f'{rootdir}/figs/last_check'
    os.makedirs(savepath, exist_ok = True)
    if os.path.exists(subdir):
        os.chdir(subdir)
        os.system(f"rm {savepath}/SDSSTG{ids[i]}.png")
        os.system(f'ds9 pnS003-fovimsky-350-2000.fits -region sources_final_sum_srcreg_fk5_edit.reg -region pn-bkgregtsky.reg -region R500_01_fk5.reg -scale log -cmap b -smooth yes -colorbar no -zoom to fit &')
        os.system(f'sleep 5')
        os.system(f"xpaset -p ds9 saveimage png {savepath}/SDSSTG{ids[i]}.png")
        os.system(f"xpaset -p ds9 exit")




10.2 make mask!

In [22]:
## make mask!
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

ids = dat_f['SRCID']
# ids = ['SDSSTG40241']

for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    f = open(f'makemask_2nd.sh', 'w')
    for name in ['mos1', 'mos2', 'pn']:
        imgfile = glob(f'{name}*-fovimsky-350-2000.fits')[0]
        mskfile = glob(f'{name}*-fovimtmask.fits')[0]
        longname = mskfile.split('-')[0]
        f.write(f'''
makemask imagefile={imgfile} maskfile={mskfile} regionfile={longname}-bkgregtsky.fits cheesefile={longname}-cheeset.fits 2>&1 | tee makemask-{name}_2nd.log
        ''')

    f.close()

100%|██████████| 45/45 [00:00<00:00, 339.17it/s]


10.2 check *-cheeset.fits!

In [20]:
rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
datapath = f'{rootdir}/dom'


with fits.open(f'{datapath}/xgap_m500_forecast.fits') as f:
    dat = f[1].data
    doc_ids = dat['ID']

all_ids = np.array([f.split('/')[-1].split('G')[-1] for f in glob(f'{rootdir}/SDSS*')]).astype(int)
# fin_ids = np.array([f.split('/')[-1].split('.')[0].split('G')[-1] for f in glob(f'{rootdir}/figs/srcs_final_resize_05/*.png')]).astype(int)
miss_ids = all_ids[~np.isin(all_ids, doc_ids)]

# ids = all_ids
ids = ['10159','15354']
for i in range(len(ids)):
    subdir = f"{rootdir}/SDSSTG{ids[i]}"
    savepath = f'{rootdir}/figs/cheeset'
    os.makedirs(savepath, exist_ok = True)
    if os.path.exists(subdir):
        os.chdir(subdir)
        for name in ['mos1', 'mos2', 'pn']:
            if glob(f'{name}-cheeset.fits'):
                cheese_file = glob(f'{name}-cheeset.fits')[0]
                os.system(f"rm {savepath}/SDSSTG{ids[i]}_{name}-cheeset.png")
                os.system(f'ds9 {cheese_file} -region {name}-bkgregtsky.reg -scale log -colorbar no -zoom to fit &')
                os.system(f'sleep 5')
                os.system(f"xpaset -p ds9 saveimage png {savepath}/SDSSTG{ids[i]}_{name}-cheeset.png")
                os.system(f"xpaset -p ds9 exit")




Fontconfig warning: ignoring UTF-8: not a valid region tag
Fontconfig warning: ignoring UTF-8: not a valid region tag
Fontconfig warning: ignoring UTF-8: not a valid region tag
Fontconfig warning: ignoring UTF-8: not a valid region tag
Fontconfig warning: ignoring UTF-8: not a valid region tag
Fontconfig warning: ignoring UTF-8: not a valid region tag


10.3 comb image!

In [ ]:
def reorder_list(input_list):
    order = ['mos1', 'mos2', 'pn']
    return sorted(input_list, key=lambda x: (order.index(x[:-4]), x[-4:]))

import os
from glob import glob

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('comb_img.sh', 'w')

        elo = 500
        ehi = 4000

        name_dir = {'mos1':'mos1*-allevc', 'mos2':'mos2*-allevc', 'pn':'pn*-allevc'}

        express = f'(PATTERN<=4)&&((FLAG & 0xb0000) == 0)&&((FLAG & 0x762a097c) == 0)&&(PI in [{elo}:{ehi}])&&(DETY in [-16510:14345])'
        plot_express = f'''imagebinning='imageSize' imagedatatype='Int32' squarepixels=yes ignorelegallimits=yes withxranges=yes withyranges=yes xcolumn='X' ximagesize=900 ximagemax=48400 ximagemin=3401 ycolumn='Y' yimagesize=900 yimagemax=48400 yimagemin=3401'''

        name_lst = []
        prefix_lst = ''
        for file in glob(f'*0*-allevc.fits'):
            prefix = file.split('-')[0]
            name_lst.append(prefix)
        name_lst = reorder_list(name_lst)
        for prefix in name_lst:
            if 'mos' in prefix:
                prefix_lst += f"{prefix[-5:]}  "
            else:
                prefix_lst += f"{prefix[-4:]}"
        print(prefix_lst)
        for name in name_lst:
            f.write(f'''source set_sas.txt \n''')
            f.write(f'''source set_chips_on.txt \n''')
            f.write(f'''evselect table={name}-allevc.fits withimageset=Y {plot_express} expression='{express}' imageset='{name}-fovimsky-{elo}-{ehi}.fits'  \n''')
            if 'pn' in name:
                f.write(f'''evselect table={name}-allevcoot.fits withimageset=Y {plot_express} expression='{express}' imageset='{name}-fovimootsky-{elo}-{ehi}.fits'  \n''')
            f.write(f'mv {name}-fovimt.fits {name}-fovimsky-{elo}-{ehi}.fits\n')
            f.write(f'cp *expimsky* {name}-expimsky-{elo}-{ehi}.fits\n')
            f.write(f'cp {name[:-4]}-cheeset.fits {name}-cheeset.fits\n')
        f.write(f'''combimage prefixlist="{prefix_lst}" withcheese=yes cheesetype=t elowlist={elo} ehighlist={ehi} > combimg.log 2>&1 \n ''')
        f.write(f'''mv comb-fovimsky-{elo}-{ehi}.fits comb-fovimsky-{elo}-{ehi}_masked.fits\n''')
        f.write(f'''mv comb-expimsky-{elo}-{ehi}.fits comb-expimsky-{elo}-{ehi}_masked.fits\n''')
        f.write(f'''combimage prefixlist="{prefix_lst}" withcheese=no elowlist={elo} ehighlist={ehi} > -a combimg.log 2>&1 \n ''')
        f.close()